In [1]:
%matplotlib inline

In [ ]:
import sys
sys.path

In [ ]:
sys.path.append('/workspaces/ennf-fork')

In [ ]:
from nn_lib.mdl.loss_functions import BCELoss_logits
from nn_lib.optim import Adam, Optimizer
from nn_lib.data import Dataloader

In [ ]:
from toy_mlp.toy_segmentation.model_trainer import UNetTrainer
from toy_mlp.toy_segmentation.small_unet_1 import SmallUNet1
from toy_mlp.toy_segmentation.PH2_dataset import PH2
from nn_lib.scheduler.multi_step_lr import MultiStepLR

In [ ]:
n_epochs=10
optim: Optimizer = Adam
batch_size=25
milestones=[2,9]

In [ ]:
# generate a training dataset
size = (16,16)
#size = (32,32)
train_dataset = PH2(ds_type='train',size=size)
# generate a validation dataset different from the training dataset
val_dataset = PH2(ds_type='val',size=size)
# create a dataloader for training data with shuffling and dropping last batch
train_dataloader = Dataloader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
# create a dataloader for validation dataset without shuffling or last batch dropping
val_dataloader = Dataloader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

In [ ]:
mlp_model = SmallUNet1(kernum=64)
print(f'Created the following binary MLP classifier:\n{mlp_model}')
# create loss function
loss_fn = BCELoss_logits() #CELoss()
# create optimizer for model parameters
optimizer = optim(mlp_model.parameters(), lr=5*1e-3, weight_decay=5e-4)
scheduler = MultiStepLR(optimizer, milestones=milestones, gamma=0.2)
# create a model trainer
model_trainer = UNetTrainer(mlp_model, loss_fn, optimizer, scheduler=scheduler)

In [ ]:
model_trainer.set_datasets(train_dataloader, val_dataloader)
model_trainer.train(20)

In [ ]:
h = model_trainer.history

In [ ]:
print(h[0])

In [ ]:
print(h[1])

In [ ]:
print(h[2])

In [ ]:
print(h[3])

In [ ]:
from matplotlib import pyplot as plt

plt.plot(h[0],label="train loss")
plt.plot(h[2],label="val loss")
plt.legend(loc='best')
plt.xlabel('epoches')
plt.ylabel('loss')
plt.show()


In [ ]:
plt.plot(h[1],label="train score")
plt.plot(h[3],label="val score")
plt.legend(loc='best')
plt.xlabel('epoches')
plt.ylabel('score')
plt.show()